## Imports and Config

In [15]:
# CELL 1: Imports and Configuration
import json
import pandas as pd
# CRITICAL FIX: The robust library to use is python-binance, imported as Client
from binance.client import Client 
# REMOVED: Imports for FuturesClient have been removed as the 'binance.futures' module is missing.
from datetime import datetime
import time  # For sleep
import requests
print(requests.get('https://api.ipify.org').text)
# NEW ADDITION: Imports for Technical Analysis (TA-Lib)
from talib import SMA, EMA, RSI, ATR, BBANDS

# Load config
try:
    with open('config.json') as f:
        config = json.load(f)
    print("✅ Configuration loaded.")
except FileNotFoundError:
    print("⚠️ WARNING: config.json not found. Ensure it exists in the root directory.")
    config = {}

# Define the Futures Testnet URL separately for clarity
FUTURES_TESTNET_URL = 'https://testnet.binancefuture.com'

# --- NEW FUNCTION TO ENSURE TESTNET CLIENT IS CREATED CORRECTLY ---
def get_testnet_client():
    """
    Initializes and returns a Binance Client connected to the Futures Testnet.
    
    CRITICAL FIX: Removed 'base_url' from Client constructor as it causes an error 
    in extremely old library versions. We rely on the client's internal routing 
    when calling futures-prefixed methods (futures_ping, futures_klines, etc.).
    """
    api_key = config.get('binance_key')
    api_secret = config.get('binance_secret')
    
    if not api_key or not api_secret:
        print("❌ Error: API key or secret missing from config.json.")
        return None

    try:
        # CRITICAL FIX: Initialize standard Client *without* base_url, as it is not supported
        client = Client(
            api_key=api_key,
            api_secret=api_secret,
            tld='com' # tld is generally supported, we keep it.
        )
        
        # Verify connectivity using a standard futures method
        client.futures_ping()
        print("✅ Binance Testnet client initialized and connected.")
        return client
    except Exception as e:
        # Check for API Key error
        if "APIError(code=-2015)" in str(e) or "Authentication failed" in str(e):
             print("❌ Could not initialize Testnet client. Error: API Key / Permission Issue. Please verify your 'binance_key' and 'binance_secret'.")
        else:
             print(f"❌ Could not initialize Testnet client. Error: {e}")
             print("\n!!! URGENT: Your python-binance library is too old and lacks critical features like 'base_url'. Please update it using: pip install --upgrade python-binance !!!")
        return None


102.215.33.116
✅ Configuration loaded.


## fetch_data

In [2]:
# CELL 2: Revised 'fetch_data' Function (Uses Binance API History Loop)
def fetch_data(symbol='BTCUSDT', interval='1d', start_date_str="2017-08-17"): 
    """
    Fetches long-term historical data (1d interval) from the public Binance API, 
    and recent 1h klines and current price from the authenticated Testnet client.
    
    Args:
        symbol (str): The trading pair (e.g., 'BTCUSDT').
        interval (str): The historical kline interval (e.g., '1d').
        start_date_str (str): The date to start fetching history from.
        
    Returns:
        dict: A dictionary containing historical_klines, recent_1h, resampled data, 
              and current_price. Returns None on failure.
    """
    # The 'config' dictionary must be defined in Cell 1 and accessible globally.
    try:
        # Initialize Client for PUBLIC (non-futures) historical data.
        # We use a client without special URL for public API endpoints
        client = Client(
            api_key=config.get('binance_key'), # Use .get for safety
            api_secret=config.get('binance_secret') # Use .get for safety
        )
        print("Binance client initialized for public data fetch.")
        
        # Initialize Testnet client for recent data and current price
        testnet_client = get_testnet_client()
        if testnet_client is None:
            return None
        
        # --- 1. Historical Data (Looping Fetch from PUBLIC API) ---
        print(f"Starting historical {interval} kline fetch for {symbol} from {start_date_str}...")
        
        start_ts = int(datetime.strptime(start_date_str, "%Y-%m-%d").timestamp() * 1000)
        now_ts = int(time.time() * 1000)
        all_data = []
        
        # Use public API for deep history 
        public_url = "https://api.binance.com/api/v3/klines"
        
        while start_ts < now_ts:
            # Fetch 1000 candles at a time
            params = {"symbol": symbol, "interval": interval, "startTime": start_ts, "limit": 1000}
            
            r = requests.get(public_url, params=params)
            r.raise_for_status() # Raise error for bad response codes (4xx, 5xx)
            chunk = r.json()
            
            if not chunk or len(chunk) < 2: 
                break
            
            all_data.extend(chunk)
            # Set next start time to last candle's open time + 1ms to avoid overlap
            start_ts = chunk[-1][0] + 1 
            
            print(f"Fetched {len(chunk)} candles, continuing from: {datetime.fromtimestamp(start_ts/1000).strftime('%Y-%m-%d')}")
            time.sleep(0.5) # Politeness delay to avoid rate limits

        print(f"Total historical candles fetched: {len(all_data)}")
        
        # Convert historical data to DataFrame
        cols = ["open_time", "open", "high", "low", "close", "volume", "close_time", 
                "quote_asset_volume", "num_trades", "taker_buy_base", "taker_buy_quote", "ignore"]
        hist_df = pd.DataFrame(all_data, columns=cols)
        
        # Clean historical data
        if not hist_df.empty:
            numeric_cols = ["open", "high", "low", "close", "volume"]
            hist_df[numeric_cols] = hist_df[numeric_cols].apply(pd.to_numeric)
            hist_df['timestamp'] = pd.to_datetime(hist_df['open_time'], unit='ms')
            # Drop unnecessary columns after conversion
            hist_df.drop(columns=['open_time', 'close_time', 'ignore', 'taker_buy_base', 'taker_buy_quote'], inplace=True)
            hist_df.set_index('timestamp', inplace=True)
        
        # --- 2. Recent 24hr Kliness (1h interval) from Testnet Client ---
        print("Fetching recent 24h klines (1h) from Testnet...")
        # Use the testnet_client created above (endpoint already changed)
        klines = testnet_client.futures_klines(
            symbol=symbol, 
            interval='1h', 
            limit=500
        )
        
        recent_df = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        
        # Clean recent data
        numeric_cols_recent = ['open', 'high', 'low', 'close', 'volume']
        recent_df[numeric_cols_recent] = recent_df[numeric_cols_recent].apply(pd.to_numeric)
        recent_df['timestamp'] = pd.to_datetime(recent_df['open_time'], unit='ms')
        recent_df.set_index('timestamp', inplace=True)
        recent_df.drop(columns=['open_time', 'close_time', 'ignore', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume'], inplace=True)


        # Resample recent data (used for quick, aggregated view)
        resampled_4h = recent_df.resample('4H').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}).dropna()
        resampled_1d = recent_df.resample('D').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}).dropna()

        # --- 3. Current Price from Testnet Client ---
        # Use the testnet_client created above (endpoint already changed)
        current_price = float(testnet_client.futures_symbol_ticker(
            symbol=symbol
        )['price'])
        print(f"Current Testnet Price: {current_price}")

        return {
            'historical_klines': hist_df,       # Full history (1d by default)
            'recent_1h': recent_df,           # Last 24 hours (1h candles)
            'resampled_4h': resampled_4h,       # 4h candles from recent data
            'resampled_1d': resampled_1d,       # 1d candle from recent data
            'current_price': current_price
        }

    except requests.HTTPError as he:
        print(f"❌ API HTTP Error: {he}. Check symbol, interval, or rate limits.")
        return None
    except Exception as e:
        print(f"❌ Data fetch error: {e}")
        return None

## The test script

In [3]:
# CELL 3: Test the Function

# Check for a 'data' folder and create it if necessary
import os
if not os.path.exists('data'):
    os.makedirs('data')
    print("Created 'data' directory.")

print("Running data fetch test...")
# Call the function. It will print its status as it fetches history.
data = fetch_data()

if data:
    print("\n------------------------------------------------------")
    print(f"✅ DATA FETCH SUCCESSFUL ✅")
    print(f"Current Price (Testnet): {data['current_price']:,}")
    print("\nRecent 1H Data Head:")
    print(data['recent_1h'].head())
    print("\nHistorical 1D Data Tail:")
    print(data['historical_klines'].tail())
    print("------------------------------------------------------")
    
    # Export historical data for later use
    data['historical_klines'].to_csv('data/BTCUSDT_historical_1d.csv')
else:
    print("❌ Data fetch failed. Check the error message above for details.")

Running data fetch test...
Binance client initialized for public data fetch.
✅ Binance Testnet client initialized and connected.
Starting historical 1d kline fetch for BTCUSDT from 2017-08-17...
Fetched 1000 candles, continuing from: 2020-05-12
Fetched 1000 candles, continuing from: 2023-02-06
Fetched 1000 candles, continuing from: 2025-11-02
Fetched 23 candles, continuing from: 2025-11-25
Total historical candles fetched: 3023
Fetching recent 24h klines (1h) from Testnet...


C:\Users\Mesh\AppData\Local\Temp\ipykernel_9584\706114319.py:95: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  resampled_4h = recent_df.resample('4H').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}).dropna()


Current Testnet Price: 86257.7

------------------------------------------------------
✅ DATA FETCH SUCCESSFUL ✅
Current Price (Testnet): 86,257.7

Recent 1H Data Head:
                         open      high       low     close     volume  \
timestamp                                                                
2025-11-05 00:00:00  101450.2  101700.0  100308.0  100522.9   8632.804   
2025-11-05 01:00:00  100523.0  100988.1   98909.7  100722.1  19826.387   
2025-11-05 02:00:00  100722.1  101918.1  100713.5  101633.7  17538.476   
2025-11-05 03:00:00  101633.8  102222.0  101405.1  102094.8   9519.477   
2025-11-05 04:00:00  102094.8  102212.3  101481.1  101952.1   6278.147   

                    quote_asset_volume  number_of_trades  
timestamp                                                 
2025-11-05 00:00:00    871549293.11530            301082  
2025-11-05 01:00:00   1977195462.46960            624322  
2025-11-05 02:00:00   1777363710.88040            542493  
2025-11-05 03:00:

In [3]:
# CELL 4: Indicator Helper Functions (NEW)

def fibonacci_levels(high, low):    
    """
    Calculates key Fibonacci retracement levels based on a given high and low range.
    """
    diff = high - low    
    return {        
        '23.6%': high - 0.236 * diff,        
        '38.2%': high - 0.382 * diff,        
        '50%': high - 0.5 * diff,        
        '61.8%': high - 0.618 * diff,    
    }

def compute_indicators(df):    
    """
    Computes a set of standard technical indicators using TA-Lib on the provided DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame containing 'close', 'high', and 'low' columns.
        
    Returns:
        dict: A dictionary of indicator values, or None on failure.
    """
    try:        
        # We assume the input DataFrame (e.g., recent_1h) is already cleaned and numeric.
        closes = df['close'].values        
        highs = df['high'].values        
        lows = df['low'].values                
        
        indicators = {
            # Simple Moving Averages
            'sma50': SMA(closes, timeperiod=50)[-1] if len(closes) >= 50 else None,
            'sma200': SMA(closes, timeperiod=200)[-1] if len(closes) >= 200 else None,
            
            # Exponential Moving Averages
            'ema12': EMA(closes, timeperiod=12)[-1] if len(closes) >= 12 else None,
            'ema26': EMA(closes, timeperiod=26)[-1] if len(closes) >= 26 else None,
            
            # Momentum and Volatility Indicators
            'rsi14': RSI(closes, timeperiod=14)[-1] if len(closes) >= 14 else None,
            'atr14': ATR(highs, lows, closes, timeperiod=14)[-1] if len(closes) >= 14 else None,
            'bollinger_upper': BBANDS(closes, timeperiod=20)[0][-1] if len(closes) >= 20 else None,
            'bollinger_middle': BBANDS(closes, timeperiod=20)[1][-1] if len(closes) >= 20 else None,
            'bollinger_lower': BBANDS(closes, timeperiod=20)[2][-1] if len(closes) >= 20 else None,
            
            # Fibonacci Retracements
            'fib_levels': fibonacci_levels(max(highs), min(lows))
        }                
        
        # Store to CSV
        # We need to flatten the nested 'fib_levels' dict before saving to CSV.
        
        # Create a flattened dictionary for CSV
        csv_indicators = {}
        for key, value in indicators.items():
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    csv_indicators[f"fib_{sub_key.replace('%', '').replace('.', '_')}"] = sub_value
            else:
                csv_indicators[key] = value

        pd.DataFrame([csv_indicators]).to_csv('data/indicators.csv', index=False)
        
        print("✅ Indicators computed and saved to data/indicators.csv")
        return indicators    
        
    except Exception as e:        
        print(f"❌ Indicator error: {e}")        
        return None


In [4]:
# CELL 3: Test the Data Fetch Function (Renumbered to 5)

# Check for a 'data' folder and create it if necessary
import os
if not os.path.exists('data'):
    os.makedirs('data')
    print("Created 'data' directory.")

print("Running data fetch test...")
# Call the function. It will print its status as it fetches history.
data = fetch_data()

if data:
    print("\n------------------------------------------------------")
    print(f"✅ DATA FETCH SUCCESSFUL ✅")
    print(f"Current Price (Testnet): {data['current_price']:,}")
    print("\nRecent 1H Data Head:")
    print(data['recent_1h'].head())
    print("\nHistorical 1D Data Tail:")
    print(data['historical_klines'].tail())
    print("------------------------------------------------------")
    
    # Export historical data for later use
    data['historical_klines'].to_csv('data/BTCUSDT_historical_1d.csv')
else:
    print("❌ Data fetch failed. Check the error message above for details.")

Running data fetch test...
Binance client initialized for public data fetch.
✅ Binance Testnet client initialized and connected.
Starting historical 1d kline fetch for BTCUSDT from 2017-08-17...
Fetched 1000 candles, continuing from: 2020-05-12
Fetched 1000 candles, continuing from: 2023-02-06
Fetched 1000 candles, continuing from: 2025-11-02
Fetched 33 candles, continuing from: 2025-12-05
Total historical candles fetched: 3033
Fetching recent 24h klines (1h) from Testnet...


C:\Users\Mesh\AppData\Local\Temp\ipykernel_14696\706114319.py:95: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  resampled_4h = recent_df.resample('4H').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}).dropna()


Current Testnet Price: 89306.1

------------------------------------------------------
✅ DATA FETCH SUCCESSFUL ✅
Current Price (Testnet): 89,306.1

Recent 1H Data Head:
                        open     high      low    close    volume  \
timestamp                                                           
2025-11-14 23:00:00  95147.5  95147.5  93955.0  94544.9  8705.186   
2025-11-15 00:00:00  94544.9  95395.3  94523.9  95120.0  6755.663   
2025-11-15 01:00:00  95120.0  95766.5  94715.5  95617.0  5772.021   
2025-11-15 02:00:00  95617.0  96474.6  95374.8  96452.5  7119.281   
2025-11-15 03:00:00  96452.4  96800.0  96100.8  96388.5  7759.499   

                    quote_asset_volume  number_of_trades  
timestamp                                                 
2025-11-14 23:00:00    822137557.42160            305635  
2025-11-15 00:00:00    642216829.68100            268455  
2025-11-15 01:00:00    550140125.94650            216368  
2025-11-15 02:00:00    682789549.13390            22

In [5]:
# CELL 6: Test the Indicator Computation (NEW)

if 'data' in globals() and data:
    print("\nRunning indicator computation test on recent 1H data...")
    # Use the 'recent_1h' data as input, which is the last 24 hours of data.
    indicators = compute_indicators(data['recent_1h'])
    
    if indicators:
        print("\n✅ Computed Indicators:")
        # Print with cleaner formatting
        for key, value in indicators.items():
            if key == 'fib_levels':
                print(f"  {key}:")
                for f_key, f_value in value.items():
                    print(f"    - {f_key}: {f_value:,.2f}")
            else:
                print(f"  {key}: {value:,.2f}" if value is not None else f"  {key}: None (Need more data points)")
else:
    print("⚠️ Skipping indicator test: Data not successfully fetched in previous step.")


Running indicator computation test on recent 1H data...
✅ Indicators computed and saved to data/indicators.csv

✅ Computed Indicators:
  sma50: 92,239.16
  sma200: 90,614.31
  ema12: 90,380.45
  ema26: 91,168.27
  rsi14: 30.05
  atr14: 801.09
  bollinger_upper: 93,475.61
  bollinger_middle: 91,244.71
  bollinger_lower: 89,013.80
  fib_levels:
    - 23.6%: 92,976.80
    - 38.2%: 90,611.60
    - 50%: 88,700.00
    - 61.8%: 86,788.40


In [6]:

# CELL 7: Strategy Definition (PHASE 3 START - Corrected NameError)

def generate_signal(indicators, current_price, atr_multiplier=2.0):
    """
    Generates a trading signal (BUY/SELL/HOLD) based on multiple indicator confirmations
    and defines Stop Loss (SL) and Take Profit (TP) levels using ATR.
    
    Strategy:
    1. Base signal on SMA crossover (Long-term trend).
    2. Confirmation by RSI (Oversold/Overbought).
    3. Final confirmation by Bollinger Bands (Reversion to mean setup).
    4. Confidence boost if near Fibonacci 61.8% level.
    
    Args:
        indicators (dict): The dictionary of calculated indicator values.
        current_price (float): The current market price.
        atr_multiplier (float): Multiplier for ATR to calculate SL/TP levels (Risk Management).
        
    Returns:
        dict: The complete signal dictionary including risk management levels.
    """
    signal = 'HOLD'
    confidence = 0.5
    rationale = "No clear signal."
    stop_loss = None
    take_profit = None
    
    # Extract necessary indicator values, using .get for safety
    rsi = indicators.get('rsi14')
    atr = indicators.get('atr14')
    bb_upper = indicators.get('bollinger_upper')
    bb_middle = indicators.get('bollinger_middle') # Ensure bb_middle is extracted
    bb_lower = indicators.get('bollinger_lower')
    sma50 = indicators.get('sma50')
    sma200 = indicators.get('sma200')
    fib_61_8 = indicators.get('fib_levels', {}).get('61.8%')

    # Data validation for essential short-term components for SL/TP
    if any(val is None for val in [rsi, atr, bb_upper, bb_middle, bb_lower, fib_61_8]):
        rationale = "Insufficient short-term data (RSI, ATR, BBands, Fib levels) to generate a signal."
        return {
            "signal": signal, 
            "confidence": 0.0, 
            "rationale": rationale, 
            "stop_loss": stop_loss, 
            "take_profit": take_profit,
            'atr14': atr
        }
        
    # Check for proximity to Fibonacci 61.8% level (within 0.5 ATR distance)
    is_near_fib_61_8 = abs(current_price - fib_61_8) < atr * 0.5
    
    # --- Check for SMA Crossover availability (required for signal) ---
    is_sma_ready = sma50 is not None and sma200 is not None

    # --- BUY Logic ---
    if is_sma_ready and sma50 > sma200 and rsi < 30 and current_price <= bb_lower:
        signal = 'BUY'
        confidence = 0.85
        rationale = (
            f"Strong BUY signal: Bullish SMA crossover (SMA50>{sma50:,.2f} > SMA200>{sma200:,.2f}), RSI is oversold at {rsi:.2f}, "
            f"and price ({current_price:,.2f}) is at or below the lower Bollinger Band ({bb_lower:,.2f})."
        )
        
        # Boost confidence if near Fibonacci support
        if is_near_fib_61_8:
            confidence = min(1.0, confidence + 0.1) 
            rationale += f" Near Fibonacci 61.8% support ({fib_61_8:,.2f})."
            
        # Risk Management for LONG position (SL = 2 ATR below, TP = 3 ATR above)
        stop_loss = current_price - (atr * atr_multiplier)
        take_profit = current_price + (atr * atr_multiplier * 1.5) # 1.5R reward

    # --- SELL Logic ---
    elif is_sma_ready and sma50 < sma200 and rsi > 70 and current_price >= bb_upper:
        signal = 'SELL'
        confidence = 0.75
        rationale = (
            f"Strong SELL signal: Bearish SMA crossover (SMA50<{sma50:,.2f} < SMA200<{sma200:,.2f}), RSI is overbought at {rsi:.2f}, "
            f"and price ({current_price:,.2f}) is at or above the upper Bollinger Band ({bb_upper:,.2f})."
        )
        
        # Boost confidence if near Fibonacci resistance
        if is_near_fib_61_8:
            confidence = min(1.0, confidence + 0.1) 
            rationale += f" Near Fibonacci 61.8% resistance ({fib_61_8:,.2f})."

        # Risk Management for SHORT position (SL = 2 ATR above, TP = 3 ATR below)
        stop_loss = current_price + (atr * atr_multiplier)
        take_profit = current_price - (atr * atr_multiplier * 1.5) # 1.5R reward

    # --- HOLD Logic (Default or if SMA Crossover is not ready) ---
    else:
        if not is_sma_ready:
            rationale = "HOLD: Strategy requires SMA 50/200 crossover which is not available in the current window."
            confidence = 0.0 # Lowest confidence if core strategy can't run
        else:
            # More specific HOLD rationale using BB middle
            # This logic now correctly uses the extracted bb_middle variable.
            if current_price > bb_middle:
                 rationale = f"HOLD: Conditions not met for BUY/SELL. Price ({current_price:,.2f}) is in the upper half of the Bollinger Bands (BB Mid: {bb_middle:,.2f}, RSI: {rsi:.2f})."
            else:
                 rationale = f"HOLD: Conditions not met for BUY/SELL. Price ({current_price:,.2f}) is in the lower half of the Bollinger Bands (BB Mid: {bb_middle:,.2f}, RSI: {rsi:.2f})."
    
    return {
        "signal": signal, 
        "confidence": confidence, 
        "rationale": rationale, 
        "stop_loss": stop_loss, 
        "take_profit": take_profit,
        # Pass ATR value for use in position sizing calculation
        'atr14': atr
    }

In [7]:

# CELL 9: Simple Backtest Function (NEW)

def simple_backtest(hist_df, indicators_func, min_data_points=200):
    """
    Performs a simple walk-forward backtest of the trading strategy on historical data.
    
    Args:
        hist_df (pd.DataFrame): The full historical kline data (e.g., 1d candles).
        indicators_func (function): The compute_indicators function.
        min_data_points (int): The minimum number of points required to start (for SMA200).
        
    Returns:
        pd.DataFrame: A DataFrame containing the generated signals and rationale.
    """
    print(f"Starting simple backtest on {len(hist_df)} historical candles. Analyzing from index {min_data_points}.")
    signals = []
    
    # Iterate through the historical data starting after the longest indicator window (SMA200)
    for i in range(min_data_points, len(hist_df)):
        # Define a sliding window of data up to the current point 'i'
        window_df = hist_df.iloc[i-min_data_points:i]
        
        # Compute indicators on the window (this will now have enough data for SMAs)
        indicators = indicators_func(window_df)
        
        # Get the closing price for the next bar (the price the signal would be executed at)
        current_price = hist_df['close'].iloc[i] 
        
        # Generate the signal
        signal_dict = generate_signal(indicators, current_price)
        
        # Add timestamp for the signal (the time of the bar we are trading on)
        signal_dict['timestamp'] = hist_df.index[i].strftime('%Y-%m-%d')
        
        signals.append(signal_dict)
        
        # NOTE: print only every 1000th iteration to avoid excessive output
        if i % 1000 == 0:
            print(f"  Processed {i} bars...")

    backtest_df = pd.DataFrame(signals)
    
    print("\nBacktest Signal Counts:")
    # Count BUY/SELL/HOLD signals
    print(backtest_df['signal'].value_counts()) 
    
    return backtest_df


In [8]:
# CELL 10: Test the Strategy and Run Simple Backtest (Updated to store signal for Cell 12)

# Ensure the indicators and current_price variables exist from Cell 5 and 6 runs
trade_signal = None # Initialize trade_signal globally
if 'indicators' in globals() and indicators and 'data' in globals() and data:
    print("\n--- Running Live Strategy Test (1H Data) ---")
    current_price = data['current_price']
    
    # Generate the signal
    trade_signal = generate_signal(indicators, current_price)
    
    print("✅ Live Trade Signal Generated:")
    print(f"  Signal: {trade_signal['signal']}")
    print(f"  Confidence: {trade_signal['confidence']:.2f}")
    print(f"  Price: {current_price:,.2f}")
    print(f"  Stop Loss: {trade_signal['stop_loss']:,.2f}" if trade_signal['stop_loss'] is not None else "  Stop Loss: N/A")
    print(f"  Take Profit: {trade_signal['take_profit']:,.2f}" if trade_signal['take_profit'] is not None else "  Take Profit: N/A")
    print(f"  Rationale: {trade_signal['rationale']}")
    
    # --- TEMPORARY OVERRIDE FOR TESTING POSITION CALCULATION ---
    # If the strategy currently returns HOLD, override it with a simulated SELL signal 
    # to test the risk management logic in Cell 11 and 12.
    if trade_signal['signal'] == 'HOLD':
        print("\n⚠️ OVERRIDE: Overriding HOLD signal to simulated SELL for Position Sizing test.")
        # Use existing ATR and current price to define a test trade
        atr = trade_signal.get('atr14', 100) # Default ATR if not available
        
        # Simulated SELL (Short) signal parameters
        trade_signal['signal'] = 'SELL'
        trade_signal['confidence'] = 0.90
        # SL: 2 ATR above current price
        trade_signal['stop_loss'] = current_price + (atr * 2.0)
        # TP: 3 ATR below current price (1.5R reward)
        trade_signal['take_profit'] = current_price - (atr * 3.0) 
        trade_signal['rationale'] = "SIMULATED SELL: FOR TESTING POSITION CALCULATION ONLY."


else:
    print("⚠️ Skipping live strategy test: Indicators or Data not successfully fetched.")
    
# --- 2. Simple Backtest (using historical 1D data) ---
if 'data' in globals() and data:
    print("\n--- Running Simple Backtest (1D Historical Data) ---")
    
    # Run the backtest using the historical 1D data
    # NOTE: The compute_indicators function will now have enough data for SMAs within the sliding window.
    backtest_results = simple_backtest(data['historical_klines'], compute_indicators)
    
    # Save the backtest results
    backtest_results.to_csv('data/backtest.csv', index=False)
    
    print("\n✅ Backtest complete.")
    print("Results saved to data/backtest.csv")
else:
    print("⚠️ Skipping backtest: Historical data not successfully fetched in previous steps.")


--- Running Live Strategy Test (1H Data) ---
✅ Live Trade Signal Generated:
  Signal: HOLD
  Confidence: 0.50
  Price: 89,306.10
  Stop Loss: N/A
  Take Profit: N/A
  Rationale: HOLD: Conditions not met for BUY/SELL. Price (89,306.10) is in the lower half of the Bollinger Bands (BB Mid: 91,244.71, RSI: 30.05).

⚠️ OVERRIDE: Overriding HOLD signal to simulated SELL for Position Sizing test.

--- Running Simple Backtest (1D Historical Data) ---
Starting simple backtest on 3033 historical candles. Analyzing from index 200.
✅ Indicators computed and saved to data/indicators.csv
✅ Indicators computed and saved to data/indicators.csv
✅ Indicators computed and saved to data/indicators.csv
✅ Indicators computed and saved to data/indicators.csv
✅ Indicators computed and saved to data/indicators.csv
✅ Indicators computed and saved to data/indicators.csv
✅ Indicators computed and saved to data/indicators.csv
✅ Indicators computed and saved to data/indicators.csv
✅ Indicators computed and saved t

In [9]:
# cell 11: 
def get_testnet_balance():
    import time, hmac, hashlib, requests, json

    with open('config.json') as f:
        config = json.load(f)

    api_key = config['binance_key']
    api_secret = config['binance_secret'].encode()

    base_url = 'https://testnet.binancefuture.com'
    endpoint = '/fapi/v2/balance'
    timestamp = int(time.time() * 1000)
    query_string = f'timestamp={timestamp}'
    signature = hmac.new(api_secret, query_string.encode(), hashlib.sha256).hexdigest()
    headers = {'X-MBX-APIKEY': api_key}
    url = f'{base_url}{endpoint}?{query_string}&signature={signature}'

    response = requests.get(url, headers=headers)
    return response.json()


def calculate_position(signal_data, current_price, max_risk_pct=0.01, leverage=10, min_qty=0.0001):
    """
    Calculates the position size based on max risk percentage and the defined Stop Loss (SL) level.
    """
    if signal_data['signal'] == 'HOLD' or signal_data.get('stop_loss') is None:
        print("Position calculation skipped: Signal is HOLD or Stop Loss is missing.")
        return None

    try:
        # ✅ Use raw balance fetch
        balance_info = get_testnet_balance()
        usdt_item = next((item for item in balance_info if item['asset'] == 'USDT'), None)
        if usdt_item is None:
            raise ValueError("USDT balance not found in futures account.")

        balance = float(usdt_item['availableBalance'])

        # 🔢 Risk calculations
        risk_capital = balance * max_risk_pct
        stop_loss = signal_data['stop_loss']
        take_profit = signal_data['take_profit']
        risk_per_unit = abs(current_price - stop_loss)

        if risk_per_unit == 0 or risk_per_unit < 0.01:
            raise ValueError(f"Risk per unit is too small ({risk_per_unit:,.2f}), cannot calculate quantity safely.")

        quantity = max(min_qty, risk_capital / risk_per_unit)
        margin = (quantity * current_price) / leverage

        print(f"Account Balance: {balance:,.2f} USDT")
        print(f"Max Risk Capital: {risk_capital:,.2f} USDT")
        print(f"Risk Per Unit (Entry to SL): {risk_per_unit:,.2f}")

        return {
            'quantity': round(quantity, 4),
            'stop_loss': round(stop_loss, 2),
            'take_profit': round(take_profit, 2),
            'margin_required': round(margin, 2),
            'estimated_risk_pct': max_risk_pct
        }

    except Exception as e:
        print(f"❌ Risk calc error: {e}")
        return None


In [10]:
# CELL 12: Test the Position Calculation (NEW)

if 'trade_signal' in globals() and trade_signal and trade_signal['signal'] != 'HOLD':
    print("\n--- Running Position Calculation Test ---")
    current_price = data['current_price']
    
    # Calculate the position
    position_data = calculate_position(trade_signal, current_price)
    
    if position_data:
        print("\n✅ Position Calculated:")
        print(f"  Signal: {trade_signal['signal']}")
        print(f"  Quantity: {position_data['quantity']:,.4f} BTC")
        print(f"  Entry Price: {current_price:,.2f}")
        print(f"  Stop Loss: {position_data['stop_loss']:,.2f}")
        print(f"  Take Profit: {position_data['take_profit']:,.2f}")
        print(f"  Margin Required (10x Lev): {position_data['margin_required']:,.2f} USDT")
        print(f"  Max Risk % per Trade: {position_data['estimated_risk_pct'] * 100:.2f}%")
    else:
        print("⚠️ Position data is None (Signal was HOLD or calculation failed).")
else:
    print("⚠️ Skipping position calculation test: No valid signal generated in Cell 10.")


--- Running Position Calculation Test ---
Account Balance: 5,608.33 USDT
Max Risk Capital: 56.08 USDT
Risk Per Unit (Entry to SL): 1,602.19

✅ Position Calculated:
  Signal: SELL
  Quantity: 0.0350 BTC
  Entry Price: 89,306.10
  Stop Loss: 90,908.29
  Take Profit: 86,902.82
  Margin Required (10x Lev): 312.61 USDT
  Max Risk % per Trade: 1.00%


In [11]:
data = fetch_data(symbol='BTCUSDT')
indicators = compute_indicators(data['recent_1h'])
current_price = data['current_price']
trade_signal = generate_signal(indicators, current_price)
print(trade_signal)

Binance client initialized for public data fetch.
✅ Binance Testnet client initialized and connected.
Starting historical 1d kline fetch for BTCUSDT from 2017-08-17...
Fetched 1000 candles, continuing from: 2020-05-12
Fetched 1000 candles, continuing from: 2023-02-06
Fetched 1000 candles, continuing from: 2025-11-02
Fetched 33 candles, continuing from: 2025-12-05
Total historical candles fetched: 3033
Fetching recent 24h klines (1h) from Testnet...


C:\Users\Mesh\AppData\Local\Temp\ipykernel_14696\706114319.py:95: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  resampled_4h = recent_df.resample('4H').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'}).dropna()


Current Testnet Price: 89205.4
✅ Indicators computed and saved to data/indicators.csv
{'signal': 'HOLD', 'confidence': 0.5, 'rationale': 'HOLD: Conditions not met for BUY/SELL. Price (89,205.40) is in the lower half of the Bollinger Bands (BB Mid: 91,239.68, RSI: 29.26).', 'stop_loss': None, 'take_profit': None, 'atr14': np.float64(801.0947114975803)}


In [24]:
@app.route('/signal', methods=['POST'])
def receive_signal():
    """
    Receives webhook data, runs the full strategy pipeline, and returns the result.
    """
    if not request.is_json:
        return jsonify({'status': 'error', 'message': 'Request must be JSON'}), 400

    webhook_data = request.json
    symbol = webhook_data.get('symbol', 'BTCUSDT').upper() # Default to BTCUSDT
    
    print(f"\n--- Processing Signal for {symbol} ---")

    # 1. Fetch Data
    market_data = fetch_data(symbol=symbol, interval='1h')
    if not market_data:
        return jsonify({'status': 'error', 'message': 'Failed to fetch market data.'}), 500

    # 2. Compute Indicators
    indicators = compute_indicators(market_data['recent_klines'])
    if not indicators:
        return jsonify({'status': 'error', 'message': 'Failed to compute indicators.'}), 500
    
    current_price = market_data['current_price']

    # 3. Generate Signal
    trade_signal = generate_signal(indicators, current_price)
    
    # 4. Calculate Position Size
    if trade_signal['signal'] != 'HOLD':
        position_data = calculate_position(trade_signal, current_price)
        
        if not position_data:
            # If position calculation fails (e.g., balance issue), we treat it as HOLD
            trade_signal['signal'] = 'HOLD'
            trade_signal['rationale'] = f"HOLD (Execution Blocked): {trade_signal['rationale']} but position calculation failed."
            print(f"⚠️ Calculation blocked execution. Final signal: HOLD.")
            final_response = trade_signal
        else:
            final_response = {**trade_signal, **position_data} # Merge dictionaries
            print(f"✅ Trade Plan Ready: {final_response['signal']} {final_response['quantity']:,} @ {current_price:,.2f}")
    else:
        final_response = trade_signal
        print(f"⚠️ Final signal: HOLD (No trade generated).")
        

NameError: name 'app' is not defined

In [72]:
def calculate_position(signal_data, current_price, max_risk_pct=0.01, leverage=10, min_qty=0.0001):
    if signal_data['signal'] == 'HOLD' or signal_data.get('stop_loss') is None:
        print("Position calculation skipped: Signal is HOLD or Stop Loss is missing.")
        return None

    try:
        # ✅ Get raw balance data
        balance_data = get_testnet_balance()
        usdt = next((item for item in balance_data if item['asset'] == 'USDT'), None)
        if usdt is None:
            raise ValueError("USDT balance not found in futures account.")

        balance = float(usdt['availableBalance'])

        # 🔢 Risk calculations
        risk_capital = balance * max_risk_pct
        stop_loss = signal_data['stop_loss']
        take_profit = signal_data['take_profit']
        risk_per_unit = abs(current_price - stop_loss)

        if risk_per_unit == 0 or risk_per_unit < 0.01:
            raise ValueError(f"Risk per unit is too small ({risk_per_unit:,.2f}), cannot calculate quantity safely.")

        quantity = max(min_qty, risk_capital / risk_per_unit)
        margin = (quantity * current_price) / leverage

        print(f"Account Balance: {balance:,.2f} USDT")
        print(f"Max Risk Capital: {risk_capital:,.2f} USDT")
        print(f"Risk Per Unit (Entry to SL): {risk_per_unit:,.2f}")

        return {
            'quantity': round(quantity, 4),
            'stop_loss': round(stop_loss, 2),
            'take_profit': round(take_profit, 2),
            'margin_required': round(margin, 2),
            'estimated_risk_pct': max_risk_pct
        }

    except Exception as e:
        print(f"❌ Risk calc error: {e}")
        return None


In [30]:
position_data = calculate_position(trade_signal, current_price)


❌ Risk calc error: name 'get_testnet_balance' is not defined


In [48]:
client = get_testnet_client()
client.futures_account_balance()


✅ Binance Testnet client initialized and connected.


BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action

In [13]:
from binance.client import Client
client = Client(api_key, api_secret)
client = get_testnet_client(api_key, api_secret)

client.futures_account_balance(base_url='https://testnet.binancefuture.com')


NameError: name 'api_key' is not defined

In [19]:
client = get_testnet_client()
if client:
    print(client.futures_account_balance(base_url='https://testnet.binancefuture.com'))


✅ Binance Testnet client initialized and connected.


BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action

In [49]:
from binance.client import Client
import json

with open('config.json') as f:
    config = json.load(f)

client = Client(config['binance_key'], config['binance_secret'])
client.FUTURES_URL = 'https://testnet.binancefuture.com'

print(client.futures_account_balance(base_url='https://testnet.binancefuture.com'))


{}


In [50]:
import json
from binance.client import Client

def test_binance_connection(config_path='config.json'):
    try:
        # Load credentials
        with open(config_path) as f:
            config = json.load(f)
        api_key = config['binance_key']
        api_secret = config['binance_secret']

        # Initialize client
        client = Client(api_key, api_secret)
        client.FUTURES_URL = 'https://testnet.binancefuture.com'

        # Test Futures balance endpoint
        balance = client.futures_account_balance(base_url='https://testnet.binancefuture.com')
        print("✅ Connection successful. Futures account balance retrieved:")
        for asset in balance:
            print(f"  {asset['asset']}: {asset['balance']} (Available: {asset.get('availableBalance', 'N/A')})")

    except Exception as e:
        if "APIError(code=-2015)" in str(e):
            print("❌ APIError -2015: Invalid API key, IP restriction, or missing permissions.")
            print("🔍 Check that your API key is from the Binance Futures Testnet, has futures permissions, and your IP is whitelisted.")
        else:
            print(f"❌ Connection failed: {e}")


In [ ]:
test_binance_connection()


In [29]:
import json
from binance.client import Client

def test_binance_connection(config_path='config.json'):
    try:
        # Load credentials
        with open(config_path) as f:
            config = json.load(f)
        api_key = config['binance_key']
        api_secret = config['binance_secret']

        # Initialize client
        client = Client(api_key, api_secret)
        client.FUTURES_URL = 'https://testnet.binancefuture.com'

        # Test Futures balance endpoint
        balance = client.futures_account_balance(base_url='https://testnet.binancefuture.com')
        print("✅ Connection successful. Futures account balance retrieved:")
        for asset in balance:
            print(f"  {asset['asset']}: {asset['balance']} (Available: {asset.get('availableBalance', 'N/A')})")

    except Exception as e:
        if "APIError(code=-2015)" in str(e):
            print("❌ APIError -2015: Invalid API key, IP restriction, or missing permissions.")
            print("🔍 Check that your API key is from the Binance Futures Testnet, has futures permissions, and your IP is whitelisted.")
        else:
            print(f"❌ Connection failed: {e}")

# Run the check
test_binance_connection()


✅ Connection successful. Futures account balance retrieved:


In [52]:
Raw balance response: []


SyntaxError: invalid syntax (4062880037.py, line 1)

In [53]:
Raw balance response: [{'asset': 'USDT', 'balance': '0.00000000', 'availableBalance': '0.00000000'}]


SyntaxError: invalid syntax (131014210.py, line 1)

In [ ]:
print("Raw balance response:", [{'asset': 'USDT', 'balance': '0.00000000', 'availableBalance': '0.00000000'}])


In [54]:
response = [{'asset': 'USDT', 'balance': '0.00000000', 'availableBalance': '0.00000000'}]
print("Raw balance response:", response)


Raw balance response: [{'asset': 'USDT', 'balance': '0.00000000', 'availableBalance': '0.00000000'}]


In [24]:
import requests
print(requests.get('https://api.ipify.org').text)


102.215.33.116


In [34]:
client.FUTURES_URL = 'https://testnet.binancefuture.com'


In [35]:
test_binance_connection()


✅ Connection successful. Futures account balance retrieved:


In [36]:
balance = client.futures_account_balance(base_url='https://testnet.binancefuture.com')
print("✅ Connection successful. Futures account balance retrieved:")
print("Raw balance response:", balance)


✅ Connection successful. Futures account balance retrieved:
Raw balance response: {}


In [59]:
from binance.client import Client

client = Client(api_key, api_secret)
client.FUTURES_URL = 'https://testnet.binancefuture.com'


In [60]:
import json

# Load credentials from config.json
with open('config.json') as f:
    config = json.load(f)

api_key = config['binance_key']
api_secret = config['binance_secret']


In [61]:
from binance.client import Client

client = Client(api_key, api_secret)
client.FUTURES_URL = 'https://testnet.binancefuture.com'

balance = client.futures_account_balance()
print("Raw balance response:", balance)


Raw balance response: {}


In [37]:
import json
from binance.client import Client

# Load credentials from config.json
with open('config.json') as f:
    config = json.load(f)

api_key = config['binance_key']
api_secret = config['binance_secret']

# Initialize Binance Futures Testnet client
client = Client(api_key, api_secret)
client.FUTURES_URL = 'https://testnet.binancefuture.com'

# Fetch and print balance
balance = client.futures_account_balance()
print("✅ Connection successful. Futures account balance retrieved:")
print("Raw balance response:", balance)


✅ Connection successful. Futures account balance retrieved:
Raw balance response: {}


In [38]:
import time
import hmac
import hashlib
import requests

# Load credentials
with open('config.json') as f:
    config = json.load(f)

api_key = config['binance_key']
api_secret = config['binance_secret'].encode()

# Prepare request
base_url = 'https://testnet.binancefuture.com'
endpoint = '/fapi/v2/balance'
timestamp = int(time.time() * 1000)
query_string = f'timestamp={timestamp}'
signature = hmac.new(api_secret, query_string.encode(), hashlib.sha256).hexdigest()
headers = {'X-MBX-APIKEY': api_key}

# Send request
url = f'{base_url}{endpoint}?{query_string}&signature={signature}'
response = requests.get(url, headers=headers)

# Show result
print("Raw response:", response.status_code)
print("Balance data:", response.json())


Raw response: 200
Balance data: [{'accountAlias': 'mYuXTimYsRTisR', 'asset': 'FDUSD', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'mYuXTimYsRTisR', 'asset': 'BFUSD', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'mYuXTimYsRTisR', 'asset': 'BNB', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'mYuXTimYsRTisR', 'asset': 'ETH', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000'

In [39]:
def get_testnet_balance():
    import time, hmac, hashlib, requests, json

    with open('config.json') as f:
        config = json.load(f)

    api_key = config['binance_key']
    api_secret = config['binance_secret'].encode()

    base_url = 'https://testnet.binancefuture.com'
    endpoint = '/fapi/v2/balance'
    timestamp = int(time.time() * 1000)
    query_string = f'timestamp={timestamp}'
    signature = hmac.new(api_secret, query_string.encode(), hashlib.sha256).hexdigest()
    headers = {'X-MBX-APIKEY': api_key}
    url = f'{base_url}{endpoint}?{query_string}&signature={signature}'

    response = requests.get(url, headers=headers)
    return response.json()


In [40]:
balance_data = get_testnet_balance()


In [41]:
usdt = next((item for item in balance_data if item['asset'] == 'USDT'), None)
if usdt:
    print(f"USDT Available: {usdt['availableBalance']}")


USDT Available: 5700.31201360


In [10]:
def get_testnet_balance():
    import time, hmac, hashlib, requests, json

    with open('config.json') as f:
        config = json.load(f)

    api_key = config['binance_key']
    api_secret = config['binance_secret'].encode()

    base_url = 'https://testnet.binancefuture.com'
    endpoint = '/fapi/v2/balance'
    timestamp = int(time.time() * 1000)
    query_string = f'timestamp={timestamp}'
    signature = hmac.new(api_secret, query_string.encode(), hashlib.sha256).hexdigest()
    headers = {'X-MBX-APIKEY': api_key}
    url = f'{base_url}{endpoint}?{query_string}&signature={signature}'

    response = requests.get(url, headers=headers)
    return response.json()


In [25]:
pip install ngrok

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.8 MB 813.7 kB/s eta 0:00:05
   ----- ---------------------------------- 0.5/3.8 MB 813.7 kB/s eta 0:00:05
   ----- ---------------------------------- 0.5/3.8 MB 813.7 kB/s eta 0:00:05
   -------- ------------------------------- 0.8/3.8 MB 545.3 kB/s eta 0:00:06
   ----------- ---------------------------- 1.0/3.8 MB 790.8 kB/s eta 0:00:04
   ----------- ---------------------------- 1.0/3.8 MB 790.8 kB/s eta 0:00:04
   ------------- -------------------------- 1.3/3.8 MB 713.5 kB/s eta 0:00:04
   ------------------- -------------------- 1.8/3.8 MB 875.9 kB/s eta 0:00:03
   ---------------------- -----